### Introduction

A URL can include either **Hypertext Transfer Protocol (HTTP)** or the **Hypertext Transfer Protocol Secure (HTTPS)**. Other types of protocols include the **File Transfer Protocol (FTP)**, **Siimple Mail Transfer (SMTP)**, and others, such as telnet, DNS, and so on.

A URL consists of the top-level domain, hostname, paths, and port of the web address.

A Lousy URLs are URLs that have been created with malicious intent. They are often the precursors to cyberattacjs that may happen in the near future. 

In [1]:
import pandas as pd
import numpy as np
import random 
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
def url_cleanse(web_url):
    web_url = web_url.lower()

    urltoken = []
    dot_slash  = []
    slash = str(web_url).split('/')
    for i in slash:
        r1 = str(i).split('-')

        token_slash = []
        for j in range(0, len(r1)):
            r2 = str(r1[j]).split('.')
            token_slash += r2 
            dot_slash += r1 + token_slash

            urltoken = list(set(dot_slash))
            if 'com' in urltoken:
                urltoken.remove('com')

        return urltoken